In [1]:
import cv2 
import numpy as np 
import math 

In [2]:
vid = cv2.VideoCapture(0) #webcam acılır

In [ ]:
while 1:
    try:
        ret ,frame = vid.read() # gelen her frame okunur
        frame = cv2.flip(frame, 1) #görüntü ters cevrilir 
        
        kernel = np.ones((3,3), np.uint8)
        
        roi = frame[100:300,100:300] # frameden belli bir alan alınır
        cv2.rectangle(frame,(100,100) , (300,300),(0,0,255),1) #frameden belli bir kısım kare içine alındı
        
        hsv = cv2.cvtColor(roi , cv2.COLOR_BGR2HSV) # belli bir alandaki bir rengi diğerlerinden sırmak için hsv kullanılır
                                                    # aldığımız görüntünün roi kısmını hsv ye cevrilir        
        
       
        # hsv görüntüsü olan görüntünün rengini alt ve üst değer aralığı girioruz
        lower_skin = np.array([0,20,70],np.uint8)
        upper_skin = np.array([20,255,255],np.uint8)
        
        mask = cv2.inRange(hsv , lower_skin , upper_skin) # görüntünün içindeki metotdun renk sınırlari içerisindeki kısmı diğerlerinden ayırmamızı sağlar
        mask = cv2.dilate(mask,kernel , iterations=4) # ayırdığımız nesnenin beyaz rengi içindeki küçük siyahlıkları beyazlastırır
        
        mask = cv2.GaussianBlur(mask ,(5,5),100) # gaus blurlama işlemi
        
        contours , _ = cv2.findContours(mask,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
        
        cnt =max(contours,key = lambda x: cv2.contourArea(x)) # en büuyük alana sahip konturu değişkene atanır
        
        epsilon = 0.0005 * cv2.arcLength(cnt,True) #0.0005 * → bu çevrenin çok küçük bir kısmı, yani ne kadar "esneklik" tanıyacağımız.
                                                    #Bu değeri büyütürsen kontur daha sade (az noktalı), küçültürsen daha detaylı olur
       
        approx = cv2.approxPolyDP(cnt,epsilon,True ) #Bu fonksiyon, bir konturu (kenarları) daha az sayıda nokta ile yaklaşık bir şekle indirger
        hull = cv2.convexHull(cnt) #konturu saran en küçük dışbükey şekli oluşturur.       
        
        areaHull = cv2.contourArea(hull) #dıs bükey zarfın alanını hesaplar
        areacontour = cv2.contourArea(cnt) #contourun alanı hesaplanır
        
        areaRatio = ((areaHull- areacontour) / areacontour )*100 #dıs bükeyın yüzde kacı contour olduğunu buluruz
        
        hull = cv2.convexHull(approx,retrunPoints = False)
        defect = cv2.convexityDefects(approx,hull) # dısbükey kusurlar alınır
         
        l = 0
        
        for i in range(defect.shape[0]):
            
            s,e,f,d = defect[i,0] #kusurlardan cekilen her listedeki 4 değer cekilir
            # cekilen değerleri contour'daki kusurun kordinatını bulmak icin contour indexi olarak verdik
            #cekilen sayılar contour indexleridir
            start =tuple(approx[s][0]) 
            end = tuple(approx[e][0])
            far =tuple(approx[f][0])

            #ücgen kenerları hesaplanır
            a = math.sqrt((end[0]-start[0])**2+(end[1]-start[1])**2)
            b = math.sqrt((far[0]-start[0])**2+(far[1]-start[1])**2)
            c= math.sqrt((end[0]-far[0])**2+(end[1] - far[1])**2)
            
            #ücgen alanı hesaplanır
            s = (a+b+c)/2
            area = math.sqrt(s*(s-a) *(s-b)*(s-c))
           
            d = (2*area) / 2  
            #parmak arasındaki acıyı bulmak için
            angle = math.acos((b**2 + c**2 - a**2) / 2*b*c)/ 57
            
            if angle <= 90 and d > 30:
                l = l+ 1
                cv2.cirle(roi , far,3,[0,0,255],-1)
                 
            cv2.line(roi,start,end,[0,0,255] , 2)
        
        l = l+1    
        font = cv2.FONT_HERSHEY_SIMLPEX           

        if l == 1: # eğer 1 hata varsa
            if areacontour < 2000: # contour alanı 200'den küçük ise
                cv2.putText(frame,"put your hands in the box",(0,50),font,2,(0,0,255),3,cv2.LİNE_AA)

            else:
                if areaRatio < 12:
                    cv2.putText(frame,"0",(0,50),font,2,(0,0,255),3,cv2.LİNE_AA)
                
                elif areaRatio < 17.5:
                    cv2.putText(frame,"best luck ",(0,50),font,2,(0,0,255),3,cv2.LİNE_AA)
                
                else:
                    cv2.putText(frame,"2",(0,50),font,2,(0,0,255),3,cv2.LİNE_AA)
                    
        elif l == 2:
            cv2.putText(frame,"2",(0,50),font,2,(0,0,255),3,cv2.LİNE_AA)
        
        elif l == 3:
            if areaRatio < 27 :
            
                cv2.putText(frame,"3",(0,50),font,2,(0,0,255),3,cv2.LİNE_AA)
            else:
                cv2.putText(frame,"okey",(0,50),font,2,(0,0,255),3,cv2.LİNE_AA)
                
        elif  l == 4:
            cv2.putText(frame,"4",(0,50),font,2,(0,0,255),3,cv2.LİNE_AA)
            
        elif l == 5:
            
            cv2.putText(frame,"5",(0,50),font,2,(0,0,255),3,cv2.LİNE_AA)    
        
        elif l == 6:
            cv2.putText(frame,"reposition",(0,50),font,2,(0,0,255),3,cv2.LİNE_AA)
        
        else:
            
            cv2.putText(frame,"reposition",(0,50),font,2,(0,0,255),3,cv2.LİNE_AA)
        
        cv2.imshow("mask",mask)
        cv2.imshow("frame",frame)
                       
    except:
        pass
    
    k = cv2.waitKey(5) & 0xFF
    if k == 27:
        break
    
vid.release()
cv2.destroyAllWindows() 
    